# Description
* Author: Seydou DIA & Simona MILADINOVA



Notebook to compute metrics with carbon footprint penalties

# Setup

In [ ]:
%store -r dic

In [ ]:
%matplotlib notebook
%load_ext autoreload

## Imports

In [ ]:
# os related
from pathlib import Path
import os 

# data related
import pandas as pd
# import missingno as msno
import datetime as dt
import numpy as np
import json


# visual related
import matplotlib.pyplot as plt
# time related

from fonctions import *

In [ ]:
import plotly.graph_objects as go

## Paths

In [ ]:

# MUST BE IN CODE FOLDER TO WORK !
PROJECT_PATH = Path(os.getcwd()).parent

RAW_DATA = PROJECT_PATH / "RAW_DATA" 
PROC_DATA = PROJECT_PATH / "PROC_DATA"

# RAW FILES
CSV_FILE = RAW_DATA / "Individual_stock_data_csv.csv"
XLSX_FILE = RAW_DATA / "Individual_stock_data.xlsx"

# PROC FILES
PROC_STOCK = PROC_DATA / "proc_stock.csv"
PROC_CARBON = PROC_DATA / "proc_carbon.csv"

STOCK_2000 = PROC_DATA / "stock_2000.csv"

STOCK_WORK = PROC_DATA / "stock_work.csv"
STOCK2USE = PROC_DATA / "stock2use.csv"
DIC2USE = PROC_DATA / "dic2use.json"


OUT_RENDEMENT = PROC_DATA / "rendement_v1.csv"

OUT_DF_CSV = PROC_DATA / "optimize_assets.csv"
OUT_DF_CSV_CARBON = PROC_DATA / "optimize_assets_carbon_penalty.csv"


# OUT_DIC_WEIGHT = PROC_DATA / "dic_weight.pickle"


DIC_METRICS_CARBON_JSON = PROC_DATA / "dic_metrics_carbon.json"

## Utils

In [ ]:
def compute_enc(weights):
    """Function that computes the ENC for given weights an a given window"""
    return 1 / sum([w**2 for w in weights])
    

def compute_ir(excess_return, track_err):
    """Function that computes the information ratio given 
    the excess return and the track_err"""
    return [ex_ret / track_err for (ex_ret, track_err) in zip(excess_return, track_err)]

def compute_di(weight, stock, vol_p):
    """Function that computes the diversification index"""
    stock_vol = stock.std() * np.sqrt(12)
    return np.dot(stock_vol, weight) / (vol_p * np.sqrt(12))

# Notebook


In [ ]:
stock_df = read_file(STOCK2USE).iloc[104:-4]

return_df = pd.read_csv(OUT_DF_CSV_CARBON)
return_df.index = read_file(STOCK2USE).index[104:-4]
return_df.drop(columns="Unnamed: 0", inplace=True)

stock_ret = stock_df.pct_change()

sp500 = read_file(STOCK_WORK)
bm_return = equally_weighted(sp500) 

return_sp = pd.DataFrame((sp500.mean(axis=1).pct_change())*100, columns=["mu"])
return_sp = return_sp.loc[return_df.index[0]:return_df.index[-1]]

In [ ]:

metrics = ["ret", "vol", "sr", 
           "track_err", "yearly_track_error",
           "ex_ret", "enc", "encb", 
           "ir", "di"]


In [ ]:
dic_carbon = dict.fromkeys(return_df.columns)
for k in dic_carbon.keys():
    dic_carbon[k] = dict.fromkeys(metrics)


In [ ]:
# Poids pour chacun des rebalancement
dic_weight = {
    pf: pd.read_csv(PROC_DATA / f"carbon_weight_{pf}.csv").rename(columns={'Unnamed: 0':"quarter"}).set_index("quarter")
    for pf in dic_carbon.keys()
}

In [ ]:
rf = 0.05

In [ ]:
for pf in dic_carbon.keys():
        
    # Rendement annualisé
    dic_carbon[pf]["ret"] = return_df[pf].mean() * 52

    # Volatilité sur les 18 ans
    dic_carbon[pf]["vol"] = return_df[pf].std() * np.sqrt(52)
    
    # Ratio de sharp
    dic_carbon[pf]["sr"] = (dic_carbon[pf]["ret"] - rf) / dic_carbon[pf]["vol"]
    
     # Tracking error
    dic_carbon[pf]["track_err"] = []
    for i in range(len(return_df)//12): # par paquet de 3 mois
        dic_carbon[pf]["track_err"].append((return_df[pf].iloc[i*12:(i+1)*12] - return_sp["mu"].iloc[i*12:(i+1)*12]).std() * np.sqrt(52))
    
    # Tracking global error
    dic_carbon[pf]["yearly_track_err"] = []
    for i in range(len(return_df)//(12*4)): # par paquet de 1 an
        dic_carbon[pf]["yearly_track_err"].append((return_df[pf].iloc[i*12*4:(i+1)*12*4] - return_sp["mu"].iloc[i*12*4:(i+1)*12*4]).std() * np.sqrt(52))
    
    # Excess return 
    dic_carbon[pf]["ex_ret"] = []
    for i in range(len(return_df)//12): # par paquet de 3 mois
        dic_carbon[pf]["ex_ret"].append( 
            52 * (return_df[pf].iloc[i*12:(i+1)*12].mean()) - 52 * (return_sp["mu"].iloc[i*12:(i+1)*12].mean())
        )
        
    # Yearly Excess return 
    dic_carbon[pf]["yearly_ex_ret"] = []
    for i in range(len(return_df)//(12*4)): # par paquet de 1 an
        dic_carbon[pf]["yearly_ex_ret"].append( 
            52 * (return_df[pf].iloc[i*12*4:(i+1)*12*4].mean()) - 52 * (return_sp["mu"].iloc[i*12*4:(i+1)*12*4].mean())
        )

    # Calcul de l'ENC
    dic_carbon[pf]["enc"] = dic_weight[pf].apply(compute_enc, axis=1)
    
    # Information ratio
    dic_carbon[pf]["ir"] = compute_ir(dic_carbon[pf]["ex_ret"], dic_carbon[pf]["track_err"])
        
    # Yearly Information ratio
    dic_carbon[pf]["yearly_ir"] = compute_ir(dic_carbon[pf]["yearly_ex_ret"],
                                             dic_carbon[pf]["yearly_track_err"])
    
    # Diversification index # récupérer les poids
    sum_di = 0
    for i in range(len(return_df)//12):
          sum_di += compute_di(
            weight=dic_weight[pf].iloc[i],
            stock=stock_ret.iloc[i*12:(i+1)*12],
            vol_p=return_df[pf].iloc[i*12:(i+1)*12].std())
            
    print(pf)
    print(sum_di)
    dic_carbon[pf]["di"] = sum_di / (len(return_df)//12)
    
         

In [ ]:
%store -r dic

In [ ]:
for pf in dic_carbon.keys():
    print(pf)
    print(dic_carbon[pf]["di"])

# Analyse

In [ ]:
plt.figure()
plt.bar(dic_carbon.keys(),
    [dic_carbon[k]["ret"] for k in dic_carbon.keys()],
       align="center",
        width=0.4
   )

plt.title("Rendement par portefeuille", fontsize=15)
plt.ylabel("%", fontsize=15)
plt.ylim(0,22)

plt.xticks(fontsize=12)
plt.yticks(fontsize=10)

In [ ]:
fig, axs = plt.subplots(1, len(dic_carbon.keys())+1, sharex=True, 
                        sharey=True,
                        figsize=(9, 6),
                        gridspec_kw={"width_ratios": [3, 3, 3, 3, 3, 1]})

dic_carbon_color = {'increasing':'green', 'decreasing':'red', 'no trend':'blue'}


labels = list(dic_carbon.keys())

for i, col in enumerate(return_df.columns):
    axs[i].hist(return_df[col], bins=50, density=True)
    axs[i].set_xlabel("%")
    axs[i].set_title(col, fontsize=11, pad=0.5)
    axs[i].grid(True)
    

    
lax = axs[-1]
lax.axis('off')
# lines = [Line2D([0], [0], color=c, linewidth=3, linestyle='-') for c in dic_carbon_color.values()]
#lax.legend(lines, labels, loc=10, fontsize=9)

# ----------------

fig.suptitle("Distribution des rendements hebdomadaires (%)")
#fig.suptitle("From {} to {})".format(start, end))
#fig.tight_layout(rect=[0,0,1,0.98])

In [ ]:
yr_idx = [str(i) for i in range(len(dic_carbon["ew"]["enc"]))]

enc_df = pd.DataFrame(index=yr_idx)

for k in list(dic_carbon.keys())[1:]:
    enc_df[k] = dic_carbon[k]["enc"].values
    
    

enc_df.plot()    
plt.title("Evolution Trimestirelle de l'ENC")
plt.xlabel("Trismestre")

In [ ]:
kpi_list = ["ret", "enc", "vol", "ex_ret", "track_err", "yearly_ir", "sr"]


In [ ]:
compare_df = pd.DataFrame(index=kpi_list, columns=["carbon", "no_carbon"])


dic_compare = {}
for k in dic.keys():
    for kpi in kpi_list:
        compare_df["no_carbon"].loc[kpi] = float(np.mean(dic[k][kpi]))
        compare_df["carbon"].loc[kpi] = float(np.mean(dic_carbon[k][kpi]))
    dic_compare[k] = compare_df.astype("float")

In [ ]:
dic_compare["maxdiv"].style.background_gradient(cmap="Blues", axis=0)

In [ ]:
compare_df = pd.DataFrame(index=["carbon_penalty", "no_carbon_penalty"], columns=kpi_list)


dic_compare = {}
for k in dic.keys():
    for kpi in kpi_list:
        compare_df[kpi].loc["no_carbon_penalty"] = float(np.mean(dic[k][kpi]))
        compare_df[kpi].loc["carbon_penalty"] = float(np.mean(dic_carbon[k][kpi]))
    dic_compare[k] = compare_df.astype("float")
    

In [ ]:
dic_compare["gmv"].style.background_gradient(cmap="Blues", axis=0)


In [ ]:
dic_compare["maxdecor"].style.background_gradient(cmap="Blues", axis=0)


In [ ]:
dic_compare["maxdiv"].style.background_gradient(cmap="Blues", axis=0)

In [ ]:
dic_compare["msr"].style.background_gradient(cmap="Blues", axis=0)